# 1 - Treat Data Base

In [ ]:
import pandas as pd
import win32com.client as win32
import pathlib

Dados_Lojas = pd.read_csv(r"Bases de Dados\Lojas.csv", encoding='latin1', sep=";")
display(Dados_Lojas)

Dados_Email = pd.read_excel(r"Bases de Dados\Emails.xlsx")
display(Dados_Email)

Dados_Vendas = pd.read_excel(r"Bases de Dados\Vendas.xlsx")
display(Dados_Vendas)

Dados_Vendas= Dados_Lojas.merge(Dados_Vendas, on="ID Loja")
display(Dados_Vendas)




In [ ]:
# Create a dictionary with the stores and their data
Dicionario_Lojas = {}
for Loja in Dados_Lojas["Loja"]:
    Dicionario_Lojas[Loja] = Dados_Vendas.loc[Dados_Vendas["Loja"]==Loja,:]
display(Dicionario_Lojas['Iguatemi Esplanada'])

# 2- Create sheet file:

In [ ]:
# Separate the most recent day of sales
Dia_mais_atual = Dados_Vendas['Data'].max()
print("{}/{}".format(Dia_mais_atual.day,Dia_mais_atual.month))

In [ ]:
caminho = pathlib.Path(r'Arquivos_CdLoja')

Pasta_Lojas = caminho.iterdir()

Lista_Nomes_ArqLojas = [arquivo.name for arquivo in Pasta_Lojas]

for Loja in Dicionario_Lojas:
    if Loja not in Lista_Nomes_ArqLojas:
        Nova_pasta = caminho / Loja
        Nova_pasta.mkdir()

    Nome_arquivo = "{}_{}_{}.xlsx".format(Dia_mais_atual.day, Dia_mais_atual.month, Loja)
    caminho_arquivo = caminho/ Loja/ Nome_arquivo
    Dicionario_Lojas[Loja].to_excel(caminho_arquivo)
        
    

# 3- Calculate Goals

In [ ]:
# Define the goals

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [ ]:
for Loja in Dicionario_Lojas:
    Dados_Loja_Expecifica = Dicionario_Lojas[Loja]
    Dados_Loja_Expecifica_dia = Dados_Loja_Expecifica.loc[Dados_Loja_Expecifica['Data']==Dia_mais_atual, :]

    Faturamento_dia = Dados_Loja_Expecifica_dia["Valor Final"].sum()

    Faturamento_ano = Dados_Loja_Expecifica["Valor Final"].sum()

    Quantidade_Prod_ano = len(Dados_Loja_Expecifica["Produto"].unique())

    Quantidade_Prod_dia = len(Dados_Loja_Expecifica_dia["Produto"].unique())

    Valor_Venda = Dados_Loja_Expecifica[["Código Venda", "Valor Final"]].groupby('Código Venda').sum()
    Ticket_medio_ano = Valor_Venda["Valor Final"].mean()

    Valor_Venda_dia = Dados_Loja_Expecifica_dia[["Código Venda", "Valor Final"]].groupby('Código Venda').sum()
    Ticket_medio_dia = Valor_Venda_dia["Valor Final"].mean()

    # Send Email
    outlook = win32.Dispatch('outlook.application')

 
    nome = Dados_Email.loc[Dados_Email['Loja']==Loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = Dados_Email.loc[Dados_Email["Loja"]==Loja, "E-mail"].values[0]
    mail.Subject = f'OnePage Dia {Dia_mais_atual.day}/{Dia_mais_atual.month} - Loja {Loja}'

    if meta_faturamento_ano > Faturamento_ano:
        cor_faturamento_ano = "red"
    else:
        cor_faturamento_ano = "green"
    if meta_faturamento_dia > Faturamento_dia:
        cor_faturamento_dia = "red"
    else:
        cor_faturamento_dia = "green"
    if meta_qtdeprodutos_ano > Quantidade_Prod_ano:
        cor_produto_ano = "red"
    else:
        cor_produto_ano = "green"
    if meta_qtdeprodutos_dia > Quantidade_Prod_dia:
        cor_produto_dia = "red"
    else:
        cor_produto_dia = "green"
    if meta_ticketmedio_ano > Ticket_medio_ano:
        cor_ticket_ano = "red"
    else:
        cor_ticket_ano = "green"
    if meta_ticketmedio_dia > Ticket_medio_dia:
        cor_ticket_dia = "red"
    else:
        cor_ticket_dia = "green" 
    
    #Email HTML
    mail.HTMLBody =f'''
        <h2><strong>OnePage Dia {Dia_mais_atual.day}/{Dia_mais_atual.month}/{Dia_mais_atual.year} - Loja {Loja}</strong></h2>
        <br>
        <p>{Dados_Email.loc[Dados_Email['Loja']==Loja, 'E-mail'].values[0]}</p>
        <br>
        <p>Bom dia, {Dados_Email.loc[Dados_Email['Loja']==Loja, 'Gerente'].values[0]}</p>
        <br>
        <p>O resultado de ontem (dia {Dia_mais_atual.day}/{Dia_mais_atual.month}) da Loja {Loja} foi:</p>
        <br>
        <table>
        <tr>
            <th>Indicador</th>
            <th>Valor Dia</th>
            <th>Meta dia</th>
            <th>Cenário Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">{Faturamento_dia:.2f}</td>
            <td style="text-align: center">{meta_faturamento_dia:.2f}</td>
            <td style="text-align: center"><font color = "{cor_faturamento_dia}">◙</font></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{Quantidade_Prod_dia:.2f}</td>
            <td style="text-align: center">{meta_qtdeprodutos_dia:.2f}</td>
            <td style="text-align: center"><font color = "{cor_produto_dia}">◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">{Ticket_medio_dia:.2f}</td>
            <td style="text-align: center">{meta_ticketmedio_dia:.2f}</td>
            <td style="text-align: center"><font color = "{cor_ticket_dia}">◙</font></td>
        </tr>
        </table>
        <br>
        <table>
         <tr>
            <th>Indicador</th>
            <th>Valor Ano</th>
            <th>Meta Ano</th>
            <th>Cenário Ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">{Faturamento_ano:.2f}</td>
            <td style="text-align: center">{meta_faturamento_ano:.2f}</td>
            <td style="text-align: center"><font color = "{cor_faturamento_ano}">◙</font></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{Quantidade_Prod_ano:.2f}</td>
            <td style="text-align: center">{meta_qtdeprodutos_ano:.2f}</td>
            <td style="text-align: center"><font color = "{cor_produto_ano}">◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">{Ticket_medio_ano:.2f}</td>
            <td style="text-align: center">{meta_ticketmedio_ano:.2f}</td>
            <td style="text-align: center"><font color = "{cor_ticket_ano}">◙</font></td>
        </tr>
        </table>
        <br>
        Att.,
        Cauan
        '''
    attachment = pathlib.Path.cwd() / caminho / Loja / f'{Dia_mais_atual.day}_{Dia_mais_atual.month}_{Loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()
    print('E-mail da Loja {} enviado'.format(Loja))


# 4- Ranking

In [ ]:
faturamento_loja_ano = Dados_Vendas[["Loja", "Valor Final"]].groupby("Loja").sum()

faturamento_loja_ano = faturamento_loja_ano.sort_values(by = "Valor Final", ascending = False)

caminho_arquivo_Ranking_ano = caminho/ "Ranking_Anual - {}_{}.xlsx".format(Dia_mais_atual.day, Dia_mais_atual.month)
faturamento_loja_ano.to_excel(caminho_arquivo_Ranking_ano)

display(faturamento_loja_ano)

Dados_Vendas_dia = Dados_Vendas.loc[Dados_Vendas["Data"] == Dia_mais_atual, :]
faturamento_loja_dia = Dados_Vendas_dia[["Loja", "Valor Final"]].groupby("Loja").sum()
faturamento_loja_dia = faturamento_loja_dia.sort_values(by = "Valor Final", ascending = False)

caminho_arquivo_Ranking_dia = caminho/ "Ranking_Dia - {}_{}.xlsx".format(Dia_mais_atual.day, Dia_mais_atual.month)
faturamento_loja_dia.to_excel(caminho_arquivo_Ranking_dia)

display(faturamento_loja_dia)

# 5- Ranking Email

In [ ]:
# Send Email
outlook = win32.Dispatch('outlook.application')


nome = Dados_Email.loc[Dados_Email['Loja']=="Diretoria", 'Gerente'].values[0]
mail = outlook.CreateItem(0)
mail.To = Dados_Email.loc[Dados_Email['Loja']=="Diretoria", 'E-mail'].values[0]
mail.Subject = f'Ranking {Dia_mais_atual.day}/{Dia_mais_atual.month}'

mail.Body = f'''
Bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_loja_dia.index[0]} com Faturamento R${faturamento_loja_dia.iloc[0, 0]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_loja_dia.index[-1]} com Faturamento R${faturamento_loja_dia.iloc[-1, 0]:.2f}

Melhor loja do Ano em Faturamento: Loja {faturamento_loja_ano.index[0]} com Faturamento R${faturamento_loja_ano.iloc[0, 0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_loja_ano.index[-1]} com Faturamento R${faturamento_loja_ano.iloc[-1, 0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
Cauan
'''

attachment = pathlib.Path.cwd() / caminho / "Ranking_Anual - {}_{}.xlsx".format(Dia_mais_atual.day, Dia_mais_atual.month)
mail.Attachments.Add(str(attachment))

attachment = pathlib.Path.cwd() / caminho / "Ranking_Dia - {}_{}.xlsx".format(Dia_mais_atual.day, Dia_mais_atual.month)
mail.Attachments.Add(str(attachment))

mail.Send()
print('E-mail do Ranking enviado')
